In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`

import $ivy.$

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

## Start

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}
import org.apache.spark.sql.functions._

In [4]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Dataframe API")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/09/06 14:50:35 INFO SparkContext: Running Spark version 3.5.0
24/09/06 14:50:35 INFO SparkContext: OS info Linux, 6.8.0-40-generic, amd64
24/09/06 14:50:35 INFO SparkContext: Java version 11.0.24
24/09/06 14:50:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/06 14:50:35 INFO ResourceUtils: ==============================================================
24/09/06 14:50:35 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/06 14:50:35 INFO ResourceUtils: ==============================================================
24/09/06 14:50:35 INFO SparkContext: Submitted application: Dataframe API
24/09/06 14:50:35 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHea

spark: SparkSession = org.apache.spark.sql.SparkSession@6fc7d508
import spark.implicits._

In [5]:
println(s"spark.version == ${spark.version}")

spark.version == 3.5.0


## 1 Создание DataFrame

### createDataFrame

#### From List

In [6]:
val data1 = Seq(("1", "Spark"), ("2", "Scala"), ("3", "Java"))

data1: Seq[(String, String)] = List(
  ("1", "Spark"),
  ("2", "Scala"),
  ("3", "Java")
)

In [7]:
val df1 = spark.createDataFrame(data1)
df1.show

24/09/06 14:50:39 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/09/06 14:50:39 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/DataFrame/spark-warehouse'.
24/09/06 14:50:40 INFO CodeGenerator: Code generated in 147.225362 ms
24/09/06 14:50:40 INFO CodeGenerator: Code generated in 16.427192 ms


+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



df1: DataFrame = [_1: string, _2: string]

In [8]:
val df1WithNames = df1.withColumnsRenamed(Map("_1" -> "StudentID", "_2" -> "Course"))
df1WithNames.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



df1WithNames: DataFrame = [StudentID: string, Course: string]

##### withSchema

In [9]:
val schema1 = StructType(Array(StructField("StudentID", StringType, false),
                              StructField("Course", StringType, true)))

val schema2 = new StructType()
                    .add(StructField("StudentID", StringType, false))
                    .add(StructField("Course", StringType, true))

val schema3 = StructType(
                StructField("StudentID", StringType, false) :: 
                StructField("Course", StringType, true) :: 
                Nil)

schema1.equals(schema2)
schema2.equals(schema3)

schema1: StructType = StructType(
  StructField("StudentID", StringType, false, {}),
  StructField("Course", StringType, true, {})
)
schema2: StructType = StructType(
  StructField("StudentID", StringType, false, {}),
  StructField("Course", StringType, true, {})
)
schema3: StructType = StructType(
  StructField("StudentID", StringType, false, {}),
  StructField("Course", StringType, true, {})
)
res9_3: Boolean = true
res9_4: Boolean = true

In [10]:
import collection.JavaConverters._

val data2 = data1.map(s => Row(s._1, s._2)).asJava
val df2 = spark.createDataFrame(data2, schema2)
df2.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



import collection.JavaConverters._
data2: java.util.List[Row] = SeqWrapper(List([1,Spark], [2,Scala], [3,Java]))
df2: DataFrame = [StudentID: string, Course: string]

#### fromRDD

In [11]:
val rdd1 = spark.sparkContext.parallelize(data1)
val fromRDD1 = spark.createDataFrame(rdd1)
fromRDD1.show

24/09/06 14:50:43 INFO CodeGenerator: Code generated in 21.785238 ms
24/09/06 14:50:43 INFO SparkContext: Starting job: show at cell11.sc:3
24/09/06 14:50:43 INFO DAGScheduler: Got job 0 (show at cell11.sc:3) with 1 output partitions
24/09/06 14:50:43 INFO DAGScheduler: Final stage: ResultStage 0 (show at cell11.sc:3)
24/09/06 14:50:43 INFO DAGScheduler: Parents of final stage: List()
24/09/06 14:50:43 INFO DAGScheduler: Missing parents: List()
24/09/06 14:50:43 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[3] at show at cell11.sc:3), which has no missing parents
24/09/06 14:50:43 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 13.0 KiB, free 4.5 GiB)
24/09/06 14:50:43 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 5.8 KiB, free 4.5 GiB)
24/09/06 14:50:43 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on ubuntu:37607 (size: 5.8 KiB, free: 4.5 GiB)
24/09/06 14:50:43 INFO SparkContext: Created

+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



rdd1: org.apache.spark.rdd.RDD[(String, String)] = ParallelCollectionRDD[0] at parallelize at cell11.sc:1
fromRDD1: DataFrame = [_1: string, _2: string]

##### withSchema

In [12]:
val rdd2 = rdd1.map(s => Row(s._1, s._2))
val fromRDD2 = spark.createDataFrame(rdd2, schema2)
fromRDD2.show

24/09/06 14:50:44 INFO CodeGenerator: Code generated in 7.967094 ms
24/09/06 14:50:44 INFO SparkContext: Starting job: show at cell12.sc:3
24/09/06 14:50:44 INFO DAGScheduler: Got job 3 (show at cell12.sc:3) with 1 output partitions
24/09/06 14:50:44 INFO DAGScheduler: Final stage: ResultStage 3 (show at cell12.sc:3)
24/09/06 14:50:44 INFO DAGScheduler: Parents of final stage: List()
24/09/06 14:50:44 INFO DAGScheduler: Missing parents: List()
24/09/06 14:50:44 INFO DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[7] at show at cell12.sc:3), which has no missing parents
24/09/06 14:50:44 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 21.4 KiB, free 4.5 GiB)
24/09/06 14:50:44 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 9.8 KiB, free 4.5 GiB)
24/09/06 14:50:44 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on ubuntu:37607 (size: 9.8 KiB, free: 4.5 GiB)
24/09/06 14:50:44 INFO SparkContext: Created 

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



rdd2: org.apache.spark.rdd.RDD[Row] = MapPartitionsRDD[4] at map at cell12.sc:1
fromRDD2: DataFrame = [StudentID: string, Course: string]

### toDF

#### From List

In [13]:
val df11 = data1.toDF()
df11.show

24/09/06 14:50:45 INFO CodeGenerator: Code generated in 16.612429 ms


+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



df11: DataFrame = [_1: string, _2: string]

In [14]:
data1.toDF("StudentID", "Course").show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



In [15]:
val columns = List("StudentID", "Course")
val df12 = data1.toDF(columns: _*)
df12.show

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



columns: List[String] = List("StudentID", "Course")
df12: DataFrame = [StudentID: string, Course: string]

#### fromRDD

In [16]:
val fromRDD11 = rdd1.toDF()
fromRDD11.show

24/09/06 14:50:46 INFO SparkContext: Starting job: show at cell16.sc:2
24/09/06 14:50:46 INFO DAGScheduler: Got job 6 (show at cell16.sc:2) with 1 output partitions
24/09/06 14:50:46 INFO DAGScheduler: Final stage: ResultStage 6 (show at cell16.sc:2)
24/09/06 14:50:46 INFO DAGScheduler: Parents of final stage: List()
24/09/06 14:50:46 INFO DAGScheduler: Missing parents: List()
24/09/06 14:50:46 INFO DAGScheduler: Submitting ResultStage 6 (MapPartitionsRDD[10] at show at cell16.sc:2), which has no missing parents
24/09/06 14:50:46 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 13.0 KiB, free 4.5 GiB)
24/09/06 14:50:46 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 5.7 KiB, free 4.5 GiB)
24/09/06 14:50:46 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on ubuntu:37607 (size: 5.7 KiB, free: 4.5 GiB)
24/09/06 14:50:46 INFO SparkContext: Created broadcast 6 from broadcast at DAGScheduler.scala:1580
24/09/06 14:5

+---+-----+
| _1|   _2|
+---+-----+
|  1|Spark|
|  2|Scala|
|  3| Java|
+---+-----+



fromRDD11: DataFrame = [_1: string, _2: string]

In [17]:
val fromRDD12 = rdd1.toDF(columns: _*)
fromRDD12.show

24/09/06 14:50:47 INFO SparkContext: Starting job: show at cell17.sc:2
24/09/06 14:50:47 INFO DAGScheduler: Got job 9 (show at cell17.sc:2) with 1 output partitions
24/09/06 14:50:47 INFO DAGScheduler: Final stage: ResultStage 9 (show at cell17.sc:2)
24/09/06 14:50:47 INFO DAGScheduler: Parents of final stage: List()
24/09/06 14:50:47 INFO DAGScheduler: Missing parents: List()
24/09/06 14:50:47 INFO DAGScheduler: Submitting ResultStage 9 (MapPartitionsRDD[13] at show at cell17.sc:2), which has no missing parents
24/09/06 14:50:47 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 13.0 KiB, free 4.5 GiB)
24/09/06 14:50:47 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 5.8 KiB, free 4.5 GiB)
24/09/06 14:50:47 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on ubuntu:37607 (size: 5.8 KiB, free: 4.5 GiB)
24/09/06 14:50:47 INFO SparkContext: Created broadcast 9 from broadcast at DAGScheduler.scala:1580
24/09/06 14:5

+---------+------+
|StudentID|Course|
+---------+------+
|        1| Spark|
|        2| Scala|
|        3|  Java|
+---------+------+



fromRDD12: DataFrame = [StudentID: string, Course: string]

### fromFile

In [18]:
val fromFile1 = spark.read.format("json").load("data/customer_data.json")
fromFile1.show(5, false)

24/09/06 14:50:47 INFO InMemoryFileIndex: It took 14 ms to list leaf files for 1 paths.
24/09/06 14:50:47 INFO InMemoryFileIndex: It took 2 ms to list leaf files for 1 paths.
24/09/06 14:50:47 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:47 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:47 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 198.5 KiB, free 4.5 GiB)
24/09/06 14:50:47 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 33.9 KiB, free 4.5 GiB)
24/09/06 14:50:47 INFO BlockManagerInfo: Added broadcast_12_piece0 in memory on ubuntu:37607 (size: 33.9 KiB, free: 4.5 GiB)
24/09/06 14:50:47 INFO SparkContext: Created broadcast 12 from load at cell18.sc:1
24/09/06 14:50:47 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:47 INFO SparkContext: Starting job: load at cell18.sc:1
24/09/06 14:50:47 INF

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|55711 Janet Plaza Apt. 865\nChristinachester, CT 62716|1988-06-21 00:15:34|Iceland       |12347     |timothy78@hotmail.com   |Katherine David|hillrachel      |
|Unit 2676 Box 9352\nDPO AA 38560                      |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, 

fromFile1: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

In [19]:
fromFile1.printSchema

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)



In [20]:
val fileSchema = StructType(
                    StructField("Address", StringType, true) ::
                    StructField("Birthdate", StringType, true) ::
                    StructField("Country", StringType, true) ::
                    StructField("CustomerID", StringType, true) ::
                    StructField("Email", StringType, true) ::
                    StructField("Name", StringType, true) ::
                    StructField("Username", StringType, true) ::
                    Nil)

fileSchema: StructType = StructType(
  StructField("Address", StringType, true, {}),
  StructField("Birthdate", StringType, true, {}),
  StructField("Country", StringType, true, {}),
  StructField("CustomerID", StringType, true, {}),
  StructField("Email", StringType, true, {}),
  StructField("Name", StringType, true, {}),
  StructField("Username", StringType, true, {})
)

In [21]:
val fromFile2 = spark.read.format("json").schema(fileSchema).load("data/customer_data.json")
fromFile2.show(5, false)

24/09/06 14:50:48 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/09/06 14:50:48 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:48 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:48 INFO MemoryStore: Block broadcast_16 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:49 INFO MemoryStore: Block broadcast_16_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:49 INFO BlockManagerInfo: Added broadcast_16_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:49 INFO SparkContext: Created broadcast 16 from show at cell21.sc:2
24/09/06 14:50:49 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:49 INFO SparkContext: Starting job: show at cell21.sc:2
24/09/06 14:50:49 INFO DAGScheduler: Got job 14 (show at cell21.sc:2) with 1 output partitions
24/09/06 14:50

+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Address                                               |Birthdate          |Country       |CustomerID|Email                   |Name           |Username        |
+------------------------------------------------------+-------------------+--------------+----------+------------------------+---------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                      |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com|Lindsay Cowan  |valenciajennifer|
|55711 Janet Plaza Apt. 865\nChristinachester, CT 62716|1988-06-21 00:15:34|Iceland       |12347     |timothy78@hotmail.com   |Katherine David|hillrachel      |
|Unit 2676 Box 9352\nDPO AA 38560                      |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com     |Leslie Martinez|serranobrian    |
|2765 Powers Meadow\nHeatherfurt, 

fromFile2: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

## 2 Основные операции

In [22]:
val customerDf = fromFile1

24/09/06 14:50:49 INFO BlockManagerInfo: Removed broadcast_10_piece0 on ubuntu:37607 in memory (size: 5.8 KiB, free: 4.5 GiB)
24/09/06 14:50:49 INFO BlockManagerInfo: Removed broadcast_7_piece0 on ubuntu:37607 in memory (size: 5.7 KiB, free: 4.5 GiB)
24/09/06 14:50:49 INFO BlockManagerInfo: Removed broadcast_17_piece0 on ubuntu:37607 in memory (size: 7.6 KiB, free: 4.5 GiB)
24/09/06 14:50:49 INFO BlockManagerInfo: Removed broadcast_14_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:49 INFO BlockManagerInfo: Removed broadcast_13_piece0 on ubuntu:37607 in memory (size: 7.4 KiB, free: 4.5 GiB)
24/09/06 14:50:49 INFO BlockManagerInfo: Removed broadcast_12_piece0 on ubuntu:37607 in memory (size: 33.9 KiB, free: 4.5 GiB)
24/09/06 14:50:49 INFO BlockManagerInfo: Removed broadcast_9_piece0 on ubuntu:37607 in memory (size: 5.8 KiB, free: 4.5 GiB)
24/09/06 14:50:49 INFO BlockManagerInfo: Removed broadcast_15_piece0 on ubuntu:37607 in memory (size: 7.6 KiB, free: 4

customerDf: DataFrame = [Address: string, Birthdate: string ... 5 more fields]

In [23]:
customerDf.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)



In [24]:
customerDf.schema

res24: StructType = StructType(
  StructField("Address", StringType, true, {}),
  StructField("Birthdate", StringType, true, {}),
  StructField("Country", StringType, true, {}),
  StructField("CustomerID", StringType, true, {}),
  StructField("Email", StringType, true, {}),
  StructField("Name", StringType, true, {}),
  StructField("Username", StringType, true, {})
)

In [25]:
customerDf.dtypes

res25: Array[(String, String)] = Array(
  ("Address", "StringType"),
  ("Birthdate", "StringType"),
  ("Country", "StringType"),
  ("CustomerID", "StringType"),
  ("Email", "StringType"),
  ("Name", "StringType"),
  ("Username", "StringType")
)

In [26]:
val h = customerDf.head

24/09/06 14:50:50 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:50 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:50 INFO MemoryStore: Block broadcast_18 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:50 INFO MemoryStore: Block broadcast_18_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:50 INFO BlockManagerInfo: Added broadcast_18_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:50 INFO SparkContext: Created broadcast 18 from head at cell26.sc:1
24/09/06 14:50:50 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:50 INFO SparkContext: Starting job: head at cell26.sc:1
24/09/06 14:50:50 INFO DAGScheduler: Got job 15 (head at cell26.sc:1) with 1 output partitions
24/09/06 14:50:50 INFO DAGScheduler: Final stage: ResultStage 15 (head at cell26.sc:1)
24/09/06 14:50

h: Row = [Unit 1047 Box 4089
DPO AA 57348,1994-02-20 00:46:27,United Kingdom,12346,cooperalexis@hotmail.com,Lindsay Cowan,valenciajennifer]

In [27]:
h(1)

res27: Any = "1994-02-20 00:46:27"

#### Select

In [28]:
customerDf.select("Birthdate", "Country").show

24/09/06 14:50:50 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:50 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:50 INFO CodeGenerator: Code generated in 5.004826 ms
24/09/06 14:50:50 INFO MemoryStore: Block broadcast_20 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:50 INFO MemoryStore: Block broadcast_20_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:50 INFO BlockManagerInfo: Added broadcast_20_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:50 INFO SparkContext: Created broadcast 20 from show at cell28.sc:1
24/09/06 14:50:50 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:50 INFO SparkContext: Starting job: show at cell28.sc:1
24/09/06 14:50:50 INFO DAGScheduler: Got job 16 (show at cell28.sc:1) with 1 output partitions
24/09/06 14:50:50 INFO DAGSchedul

+-------------------+--------------+
|          Birthdate|       Country|
+-------------------+--------------+
|1994-02-20 00:46:27|United Kingdom|
|1988-06-21 00:15:34|       Iceland|
|1974-11-26 15:30:20|       Finland|
|1977-05-06 23:57:35|         Italy|
|1996-09-13 19:14:27|        Norway|
|1969-06-21 03:39:20|        Norway|
|1993-02-25 18:37:29|       Bahrain|
|1993-03-13 12:37:34|         Spain|
|1972-11-10 12:01:08|       Bahrain|
|1973-01-13 17:17:26|      Portugal|
|1989-11-24 17:12:54|   Switzerland|
|1977-06-19 22:35:52|       Austria|
|1983-09-21 05:22:18|        Cyprus|
|1980-10-28 17:25:59|       Austria|
|1982-09-01 09:12:57|       Belgium|
|1979-02-03 03:42:47|       Belgium|
|1974-12-21 13:27:20|   Unspecified|
|1990-07-17 15:47:12|       Belgium|
|1981-07-10 00:35:00|        Cyprus|
|1989-12-26 00:58:01|       Denmark|
+-------------------+--------------+
only showing top 20 rows



In [29]:
customerDf.select(col("Country")).show

24/09/06 14:50:51 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:51 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:51 INFO CodeGenerator: Code generated in 6.349976 ms
24/09/06 14:50:51 INFO MemoryStore: Block broadcast_22 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:51 INFO MemoryStore: Block broadcast_22_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:51 INFO BlockManagerInfo: Added broadcast_22_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:51 INFO SparkContext: Created broadcast 22 from show at cell29.sc:1
24/09/06 14:50:51 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:51 INFO SparkContext: Starting job: show at cell29.sc:1
24/09/06 14:50:51 INFO DAGScheduler: Got job 17 (show at cell29.sc:1) with 1 output partitions
24/09/06 14:50:51 INFO DAGSchedul

+--------------+
|       Country|
+--------------+
|United Kingdom|
|       Iceland|
|       Finland|
|         Italy|
|        Norway|
|        Norway|
|       Bahrain|
|         Spain|
|       Bahrain|
|      Portugal|
|   Switzerland|
|       Austria|
|        Cyprus|
|       Austria|
|       Belgium|
|       Belgium|
|   Unspecified|
|       Belgium|
|        Cyprus|
|       Denmark|
+--------------+
only showing top 20 rows



In [30]:
customerDf.selectExpr("Birthdate as Date").show

24/09/06 14:50:51 INFO BlockManagerInfo: Removed broadcast_23_piece0 on ubuntu:37607 in memory (size: 7.0 KiB, free: 4.5 GiB)
24/09/06 14:50:51 INFO BlockManagerInfo: Removed broadcast_20_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:51 INFO BlockManagerInfo: Removed broadcast_19_piece0 on ubuntu:37607 in memory (size: 6.0 KiB, free: 4.5 GiB)
24/09/06 14:50:51 INFO BlockManagerInfo: Removed broadcast_21_piece0 on ubuntu:37607 in memory (size: 7.1 KiB, free: 4.5 GiB)
24/09/06 14:50:51 INFO BlockManagerInfo: Removed broadcast_16_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:51 INFO BlockManagerInfo: Removed broadcast_18_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:51 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:51 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:51 INFO MemoryStore: Block broadcast_24 stored as values in memory (estimated size 198.4 KiB, free 4.5 

+-------------------+
|               Date|
+-------------------+
|1994-02-20 00:46:27|
|1988-06-21 00:15:34|
|1974-11-26 15:30:20|
|1977-05-06 23:57:35|
|1996-09-13 19:14:27|
|1969-06-21 03:39:20|
|1993-02-25 18:37:29|
|1993-03-13 12:37:34|
|1972-11-10 12:01:08|
|1973-01-13 17:17:26|
|1989-11-24 17:12:54|
|1977-06-19 22:35:52|
|1983-09-21 05:22:18|
|1980-10-28 17:25:59|
|1982-09-01 09:12:57|
|1979-02-03 03:42:47|
|1974-12-21 13:27:20|
|1990-07-17 15:47:12|
|1981-07-10 00:35:00|
|1989-12-26 00:58:01|
+-------------------+
only showing top 20 rows



In [31]:
customerDf.withColumn("Flag", lit(true)).show

24/09/06 14:50:51 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:51 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:51 INFO CodeGenerator: Code generated in 9.595799 ms
24/09/06 14:50:51 INFO MemoryStore: Block broadcast_26 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:51 INFO MemoryStore: Block broadcast_26_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:51 INFO BlockManagerInfo: Added broadcast_26_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:51 INFO SparkContext: Created broadcast 26 from show at cell31.sc:1
24/09/06 14:50:51 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:51 INFO SparkContext: Starting job: show at cell31.sc:1
24/09/06 14:50:51 INFO DAGScheduler: Got job 19 (show at cell31.sc:1) with 1 output partitions
24/09/06 14:50:51 INFO DAGSchedul

+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+----+
|             Address|          Birthdate|       Country|CustomerID|               Email|             Name|        Username|Flag|
+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+----+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|    Lindsay Cowan|valenciajennifer|true|
|55711 Janet Plaza...|1988-06-21 00:15:34|       Iceland|     12347|timothy78@hotmail...|  Katherine David|      hillrachel|true|
|Unit 2676 Box 935...|1974-11-26 15:30:20|       Finland|     12348| tcrawford@gmail.com|  Leslie Martinez|    serranobrian|true|
|2765 Powers Meado...|1977-05-06 23:57:35|         Italy|     12349|  dustin37@yahoo.com|    Brad Cardenas|   charleshudson|true|
|17677 Mark Crest\...|1996-09-13 19:14:27|        Norway|     12350|amyholland@yahoo.com| 

In [32]:
val dfRenamed = customerDf.withColumnRenamed("Birthdate", "Date")
dfRenamed.show

24/09/06 14:50:52 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:52 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:52 INFO MemoryStore: Block broadcast_28 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:52 INFO MemoryStore: Block broadcast_28_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:52 INFO BlockManagerInfo: Added broadcast_28_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:52 INFO SparkContext: Created broadcast 28 from show at cell32.sc:2
24/09/06 14:50:52 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:52 INFO SparkContext: Starting job: show at cell32.sc:2
24/09/06 14:50:52 INFO DAGScheduler: Got job 20 (show at cell32.sc:2) with 1 output partitions
24/09/06 14:50:52 INFO DAGScheduler: Final stage: ResultStage 20 (show at cell32.sc:2)
24/09/06 14:50

+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|             Address|               Date|       Country|CustomerID|               Email|             Name|        Username|
+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|    Lindsay Cowan|valenciajennifer|
|55711 Janet Plaza...|1988-06-21 00:15:34|       Iceland|     12347|timothy78@hotmail...|  Katherine David|      hillrachel|
|Unit 2676 Box 935...|1974-11-26 15:30:20|       Finland|     12348| tcrawford@gmail.com|  Leslie Martinez|    serranobrian|
|2765 Powers Meado...|1977-05-06 23:57:35|         Italy|     12349|  dustin37@yahoo.com|    Brad Cardenas|   charleshudson|
|17677 Mark Crest\...|1996-09-13 19:14:27|        Norway|     12350|amyholland@yahoo.com|     Natalie Ford| gregoryharrison|


dfRenamed: DataFrame = [Address: string, Date: string ... 5 more fields]

In [33]:
customerDf.schema
dfRenamed.schema

res33_0: StructType = StructType(
  StructField("Address", StringType, true, {}),
  StructField("Birthdate", StringType, true, {}),
  StructField("Country", StringType, true, {}),
  StructField("CustomerID", StringType, true, {}),
  StructField("Email", StringType, true, {}),
  StructField("Name", StringType, true, {}),
  StructField("Username", StringType, true, {})
)
res33_1: StructType = StructType(
  StructField("Address", StringType, true, {}),
  StructField("Date", StringType, true, {}),
  StructField("Country", StringType, true, {}),
  StructField("CustomerID", StringType, true, {}),
  StructField("Email", StringType, true, {}),
  StructField("Name", StringType, true, {}),
  StructField("Username", StringType, true, {})
)

#### Filter

In [34]:
customerDf.filter("Country = 'Norway'").show

24/09/06 14:50:52 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Country),EqualTo(Country,Norway)
24/09/06 14:50:52 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Country#178),(Country#178 = Norway)
24/09/06 14:50:52 INFO CodeGenerator: Code generated in 9.89809 ms
24/09/06 14:50:52 INFO MemoryStore: Block broadcast_30 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:52 INFO MemoryStore: Block broadcast_30_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:52 INFO BlockManagerInfo: Added broadcast_30_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:52 INFO SparkContext: Created broadcast 30 from show at cell34.sc:1
24/09/06 14:50:52 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:52 INFO SparkContext: Starting job: show at cell34.sc:1
24/09/06 14:50:52 INFO DAGScheduler: Got j

+--------------------+-------------------+-------+----------+--------------------+----------------+---------------+
|             Address|          Birthdate|Country|CustomerID|               Email|            Name|       Username|
+--------------------+-------------------+-------+----------+--------------------+----------------+---------------+
|17677 Mark Crest\...|1996-09-13 19:14:27| Norway|     12350|amyholland@yahoo.com|    Natalie Ford|gregoryharrison|
|50047 Smith Point...|1969-06-21 03:39:20| Norway|     12352| vcarter@hotmail.com|     Dana Clarke|         hmyers|
|0297 Jacob Ranch ...|1990-06-01 14:49:52| Norway|     12381|douglaschavez@hot...|   Matthew Jones|    stephanie68|
|3102 Hopkins Walk...|1976-06-19 08:10:24| Norway|     12430|crystalromero@hot...|       Lisa Tate|       pgilbert|
|637 Philip Lock S...|1984-06-06 09:36:14| Norway|     12432|jessica87@hotmail...|  Cheryl Herring|mathewsnicholas|
|546 Tyler Prairie...|1985-05-27 10:39:47| Norway|     12433|mariahmcphe

In [35]:
customerDf.where($"Country" =!= "Iceland").show(false)

24/09/06 14:50:53 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Country),Not(EqualTo(Country,Iceland))
24/09/06 14:50:53 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Country#178),NOT (Country#178 = Iceland)
24/09/06 14:50:53 INFO CodeGenerator: Code generated in 8.789156 ms
24/09/06 14:50:53 INFO MemoryStore: Block broadcast_32 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:53 INFO MemoryStore: Block broadcast_32_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:53 INFO BlockManagerInfo: Added broadcast_32_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:53 INFO SparkContext: Created broadcast 32 from show at cell35.sc:1
24/09/06 14:50:53 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:53 INFO SparkContext: Starting job: show at cell35.sc:1
24/09/06 14:50:53 INFO DAGSche

+-----------------------------------------------------+-------------------+--------------+----------+--------------------------+-----------------+----------------+
|Address                                              |Birthdate          |Country       |CustomerID|Email                     |Name             |Username        |
+-----------------------------------------------------+-------------------+--------------+----------+--------------------------+-----------------+----------------+
|Unit 1047 Box 4089\nDPO AA 57348                     |1994-02-20 00:46:27|United Kingdom|12346     |cooperalexis@hotmail.com  |Lindsay Cowan    |valenciajennifer|
|Unit 2676 Box 9352\nDPO AA 38560                     |1974-11-26 15:30:20|Finland       |12348     |tcrawford@gmail.com       |Leslie Martinez  |serranobrian    |
|2765 Powers Meadow\nHeatherfurt, CT 53165            |1977-05-06 23:57:35|Italy         |12349     |dustin37@yahoo.com        |Brad Cardenas    |charleshudson   |
|17677 Mark Cres

#### Sort

In [36]:
customerDf.sort(col("CustomerID").desc).show

24/09/06 14:50:53 INFO BlockManagerInfo: Removed broadcast_28_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:53 INFO BlockManagerInfo: Removed broadcast_24_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:53 INFO BlockManagerInfo: Removed broadcast_30_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:53 INFO BlockManagerInfo: Removed broadcast_22_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:53 INFO BlockManagerInfo: Removed broadcast_31_piece0 on ubuntu:37607 in memory (size: 7.9 KiB, free: 4.5 GiB)
24/09/06 14:50:53 INFO BlockManagerInfo: Removed broadcast_26_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:53 INFO BlockManagerInfo: Removed broadcast_29_piece0 on ubuntu:37607 in memory (size: 7.6 KiB, free: 4.5 GiB)
24/09/06 14:50:53 INFO BlockManagerInfo: Removed broadcast_27_piece0 on ubuntu:37607 in memory (size: 7.6 KiB, fr

+--------------------+-------------------+--------------+----------+--------------------+--------------------+-----------------+
|             Address|          Birthdate|       Country|CustomerID|               Email|                Name|         Username|
+--------------------+-------------------+--------------+----------+--------------------+--------------------+-----------------+
|6942 Connie Skywa...|1973-10-24 00:52:10|United Kingdom|     12989| amber97@hotmail.com|   Brandon Contreras|           ecasey|
|79375 David Neck\...|1971-05-04 22:20:10|United Kingdom|     12988|   erica98@gmail.com|      Gabriel Romero|          qknight|
|00881 West Flat\n...|1997-03-05 19:20:57|United Kingdom|     12987|    vkeith@yahoo.com|Christopher Lawrence|        smcintyre|
|499 Jonathan Stre...|1987-10-24 20:05:15|United Kingdom|     12985| fredsmith@yahoo.com|        Xavier Myers|stricklandjeffery|
|9505 Melissa Stre...|1975-09-22 15:21:58|United Kingdom|     12984|scottjonathan@yah...|        

In [37]:
customerDf.orderBy("CustomerID").show

24/09/06 14:50:53 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:53 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:53 INFO CodeGenerator: Code generated in 4.968198 ms
24/09/06 14:50:54 INFO MemoryStore: Block broadcast_36 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:54 INFO MemoryStore: Block broadcast_36_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:54 INFO BlockManagerInfo: Added broadcast_36_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:54 INFO SparkContext: Created broadcast 36 from show at cell37.sc:1
24/09/06 14:50:54 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:54 INFO SparkContext: Starting job: show at cell37.sc:1
24/09/06 14:50:54 INFO DAGScheduler: Got job 24 (show at cell37.sc:1) with 1 output partitions
24/09/06 14:50:54 INFO DAGSchedul

+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|             Address|          Birthdate|       Country|CustomerID|               Email|             Name|        Username|
+--------------------+-------------------+--------------+----------+--------------------+-----------------+----------------+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|    Lindsay Cowan|valenciajennifer|
|55711 Janet Plaza...|1988-06-21 00:15:34|       Iceland|     12347|timothy78@hotmail...|  Katherine David|      hillrachel|
|Unit 2676 Box 935...|1974-11-26 15:30:20|       Finland|     12348| tcrawford@gmail.com|  Leslie Martinez|    serranobrian|
|2765 Powers Meado...|1977-05-06 23:57:35|         Italy|     12349|  dustin37@yahoo.com|    Brad Cardenas|   charleshudson|
|17677 Mark Crest\...|1996-09-13 19:14:27|        Norway|     12350|amyholland@yahoo.com|     Natalie Ford| gregoryharrison|


#### Repartition

In [38]:
val repartitionedDf = customerDf.repartition(5, col("Country"))
println(s"Old num partitions: ${customerDf.rdd.getNumPartitions}")
println(s"New num partitions: ${repartitionedDf.rdd.getNumPartitions}")

24/09/06 14:50:54 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:54 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:54 INFO MemoryStore: Block broadcast_38 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:54 INFO MemoryStore: Block broadcast_38_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:54 INFO BlockManagerInfo: Added broadcast_38_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:54 INFO SparkContext: Created broadcast 38 from rdd at cell38.sc:2
24/09/06 14:50:54 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.


Old num partitions: 1


24/09/06 14:50:54 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:54 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:54 INFO MemoryStore: Block broadcast_39 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:54 INFO MemoryStore: Block broadcast_39_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:54 INFO BlockManagerInfo: Added broadcast_39_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:54 INFO SparkContext: Created broadcast 39 from rdd at cell38.sc:3
24/09/06 14:50:54 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:54 INFO DAGScheduler: Registering RDD 72 (rdd at cell38.sc:3) as input to shuffle 0
24/09/06 14:50:54 INFO DAGScheduler: Got map stage job 25 (rdd at cell38.sc:3) with 1 output partitions
24/09/06 14:50:54 INFO DAGScheduler: Final stage: ShuffleMapStage

New num partitions: 5


repartitionedDf: Dataset[Row] = [Address: string, Birthdate: string ... 5 more fields]

In [39]:
println(s"Num partitions after coalesce: ${repartitionedDf.coalesce(1).rdd.getNumPartitions}")

24/09/06 14:50:54 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:54 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:54 INFO MemoryStore: Block broadcast_41 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:54 INFO MemoryStore: Block broadcast_41_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:54 INFO BlockManagerInfo: Added broadcast_41_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:54 INFO SparkContext: Created broadcast 41 from rdd at cell39.sc:1
24/09/06 14:50:54 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:54 INFO DAGScheduler: Registering RDD 79 (rdd at cell39.sc:1) as input to shuffle 1
24/09/06 14:50:54 INFO DAGScheduler: Got map stage job 26 (rdd at cell39.sc:1) with 1 output partitions
24/09/06 14:50:54 INFO DAGScheduler: Final stage: ShuffleMapStage

Num partitions after coalesce: 1


### functions

In [40]:
val datDf = customerDf.select($"Birthdate", date_format(col("Birthdate"), "yyyy-MM-dd").alias("bd"))
datDf.show(5)

24/09/06 14:50:55 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:55 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:55 INFO CodeGenerator: Code generated in 14.498205 ms
24/09/06 14:50:55 INFO MemoryStore: Block broadcast_43 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:55 INFO MemoryStore: Block broadcast_43_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:55 INFO BlockManagerInfo: Added broadcast_43_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:55 INFO SparkContext: Created broadcast 43 from show at cell40.sc:2
24/09/06 14:50:55 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:55 INFO SparkContext: Starting job: show at cell40.sc:2
24/09/06 14:50:55 INFO DAGScheduler: Got job 27 (show at cell40.sc:2) with 1 output partitions
24/09/06 14:50:55 INFO DAGSchedu

+-------------------+----------+
|          Birthdate|        bd|
+-------------------+----------+
|1994-02-20 00:46:27|1994-02-20|
|1988-06-21 00:15:34|1988-06-21|
|1974-11-26 15:30:20|1974-11-26|
|1977-05-06 23:57:35|1977-05-06|
|1996-09-13 19:14:27|1996-09-13|
+-------------------+----------+
only showing top 5 rows



datDf: DataFrame = [Birthdate: string, bd: string]

In [41]:
datDf.printSchema

root
 |-- Birthdate: string (nullable = true)
 |-- bd: string (nullable = true)



In [42]:
customerDf.withColumn("Identity", array($"Name", $"Username")).printSchema

24/09/06 14:50:55 INFO BlockManagerInfo: Removed broadcast_40_piece0 on ubuntu:37607 in memory (size: 7.3 KiB, free: 4.5 GiB)
24/09/06 14:50:55 INFO BlockManagerInfo: Removed broadcast_37_piece0 on ubuntu:37607 in memory (size: 7.0 KiB, free: 4.5 GiB)
24/09/06 14:50:55 INFO BlockManagerInfo: Removed broadcast_34_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:55 INFO BlockManagerInfo: Removed broadcast_35_piece0 on ubuntu:37607 in memory (size: 7.0 KiB, free: 4.5 GiB)
24/09/06 14:50:55 INFO BlockManagerInfo: Removed broadcast_42_piece0 on ubuntu:37607 in memory (size: 7.3 KiB, free: 4.5 GiB)
24/09/06 14:50:55 INFO BlockManagerInfo: Removed broadcast_44_piece0 on ubuntu:37607 in memory (size: 7.7 KiB, free: 4.5 GiB)
24/09/06 14:50:55 INFO BlockManagerInfo: Removed broadcast_36_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:55 INFO BlockManagerInfo: Removed broadcast_32_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)
 |-- Identity: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [43]:
customerDf
    .withColumn("Identity", array($"Name", $"Username"))
    .select("Name", "Username", "Identity")
    .show(5, false)

24/09/06 14:50:55 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:55 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:56 INFO CodeGenerator: Code generated in 7.96632 ms
24/09/06 14:50:56 INFO MemoryStore: Block broadcast_45 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:56 INFO MemoryStore: Block broadcast_45_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:56 INFO BlockManagerInfo: Added broadcast_45_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:56 INFO SparkContext: Created broadcast 45 from show at cell43.sc:4
24/09/06 14:50:56 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:56 INFO SparkContext: Starting job: show at cell43.sc:4
24/09/06 14:50:56 INFO DAGScheduler: Got job 28 (show at cell43.sc:4) with 1 output partitions
24/09/06 14:50:56 INFO DAGSchedule

+---------------+----------------+---------------------------------+
|Name           |Username        |Identity                         |
+---------------+----------------+---------------------------------+
|Lindsay Cowan  |valenciajennifer|[Lindsay Cowan, valenciajennifer]|
|Katherine David|hillrachel      |[Katherine David, hillrachel]    |
|Leslie Martinez|serranobrian    |[Leslie Martinez, serranobrian]  |
|Brad Cardenas  |charleshudson   |[Brad Cardenas, charleshudson]   |
|Natalie Ford   |gregoryharrison |[Natalie Ford, gregoryharrison]  |
+---------------+----------------+---------------------------------+
only showing top 5 rows



## 3 Агрегирование

In [44]:
customerDf.groupBy("Country").count().orderBy("count").show

24/09/06 14:50:56 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:56 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:56 INFO CodeGenerator: Code generated in 35.974706 ms
24/09/06 14:50:56 INFO MemoryStore: Block broadcast_47 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:56 INFO MemoryStore: Block broadcast_47_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:56 INFO BlockManagerInfo: Added broadcast_47_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:56 INFO SparkContext: Created broadcast 47 from show at cell44.sc:1
24/09/06 14:50:56 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:56 INFO DAGScheduler: Registering RDD 96 (show at cell44.sc:1) as input to shuffle 2
24/09/06 14:50:56 INFO DAGScheduler: Got map stage job 29 (show at cell44.sc:1) with 1 output part

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|           Singapore|    1|
|                 RSA|    1|
|             Iceland|    1|
|        Saudi Arabia|    1|
|United Arab Emirates|    1|
|      Czech Republic|    1|
|              Brazil|    1|
|             Lebanon|    1|
|              Greece|    2|
|         Unspecified|    2|
|             Bahrain|    2|
|              Israel|    4|
|                 USA|    4|
|              Poland|    6|
|              Sweden|    7|
|              Cyprus|    7|
|             Denmark|    8|
|               Japan|    8|
|           Australia|    8|
|         Netherlands|    8|
+--------------------+-----+
only showing top 20 rows



In [45]:
customerDf
    .groupBy($"Country", date_format(col("Birthdate"), "yyyy-MM-dd").alias("bd"))
    .agg(min("CustomerID"), max("CustomerID"))
    .orderBy("Country")
    .show

24/09/06 14:50:57 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:57 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:57 INFO CodeGenerator: Code generated in 13.765544 ms
24/09/06 14:50:57 INFO MemoryStore: Block broadcast_50 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:57 INFO MemoryStore: Block broadcast_50_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:57 INFO BlockManagerInfo: Added broadcast_50_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:57 INFO SparkContext: Created broadcast 50 from show at cell45.sc:4
24/09/06 14:50:57 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:57 INFO DAGScheduler: Registering RDD 105 (show at cell45.sc:4) as input to shuffle 3
24/09/06 14:50:57 INFO DAGScheduler: Got map stage job 31 (show at cell45.sc:4) with 1 output par

+---------+----------+---------------+---------------+
|  Country|        bd|min(CustomerID)|max(CustomerID)|
+---------+----------+---------------+---------------+
|Australia|1966-09-17|          12422|          12422|
|Australia|1967-11-29|          12424|          12424|
|Australia|1985-12-30|          12386|          12386|
|Australia|1986-08-28|          12434|          12434|
|Australia|1987-02-26|          12393|          12393|
|Australia|1989-08-28|          12415|          12415|
|Australia|1993-01-31|          12431|          12431|
|Australia|1993-04-02|          12388|          12388|
|  Austria|1970-08-06|          12865|          12865|
|  Austria|1973-07-16|          12818|          12818|
|  Austria|1973-12-21|          12370|          12370|
|  Austria|1976-06-13|          12374|          12374|
|  Austria|1976-09-03|          12373|          12373|
|  Austria|1977-06-19|          12358|          12358|
|  Austria|1977-11-30|          12453|          12453|
|  Austria

## 4 Операции над несколькими датафреймами

### Union

In [46]:
customerDf.count

24/09/06 14:50:57 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:57 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:57 INFO CodeGenerator: Code generated in 4.50848 ms
24/09/06 14:50:57 INFO MemoryStore: Block broadcast_53 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:57 INFO MemoryStore: Block broadcast_53_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:57 INFO BlockManagerInfo: Added broadcast_53_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:57 INFO SparkContext: Created broadcast 53 from count at cell46.sc:1
24/09/06 14:50:57 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:57 INFO DAGScheduler: Registering RDD 114 (count at cell46.sc:1) as input to shuffle 4
24/09/06 14:50:57 INFO DAGScheduler: Got map stage job 33 (count at cell46.sc:1) with 1 output pa

res46: Long = 507L

In [47]:
customerDf.union(customerDf).count

24/09/06 14:50:57 INFO BlockManagerInfo: Removed broadcast_51_piece0 on ubuntu:37607 in memory (size: 16.4 KiB, free: 4.5 GiB)
24/09/06 14:50:57 INFO BlockManagerInfo: Removed broadcast_43_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:57 INFO BlockManagerInfo: Removed broadcast_46_piece0 on ubuntu:37607 in memory (size: 7.7 KiB, free: 4.5 GiB)
24/09/06 14:50:57 INFO BlockManagerInfo: Removed broadcast_45_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:57 INFO BlockManagerInfo: Removed broadcast_49_piece0 on ubuntu:37607 in memory (size: 19.4 KiB, free: 4.5 GiB)
24/09/06 14:50:57 INFO BlockManagerInfo: Removed broadcast_54_piece0 on ubuntu:37607 in memory (size: 8.3 KiB, free: 4.5 GiB)
24/09/06 14:50:57 INFO BlockManagerInfo: Removed broadcast_48_piece0 on ubuntu:37607 in memory (size: 17.9 KiB, free: 4.5 GiB)
24/09/06 14:50:57 INFO BlockManagerInfo: Removed broadcast_47_piece0 on ubuntu:37607 in memory (size: 34.0 KiB, f

res47: Long = 1014L

In [48]:
customerDf.union(customerDf).union(customerDf).count

24/09/06 14:50:58 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:58 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:58 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:58 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:58 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:58 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:58 INFO MemoryStore: Block broadcast_60 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:58 INFO MemoryStore: Block broadcast_60_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:58 INFO BlockManagerInfo: Added broadcast_60_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:58 INFO SparkContext: Created broadcast 60 from count at cell48.sc:1
24/09/06 14:50:58 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:58 IN

res48: Long = 1521L

In [49]:
customerDf.count * 5

24/09/06 14:50:58 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:58 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:58 INFO MemoryStore: Block broadcast_65 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:58 INFO MemoryStore: Block broadcast_65_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:58 INFO BlockManagerInfo: Added broadcast_65_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:50:58 INFO SparkContext: Created broadcast 65 from count at cell49.sc:1
24/09/06 14:50:58 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:58 INFO DAGScheduler: Registering RDD 143 (count at cell49.sc:1) as input to shuffle 7
24/09/06 14:50:58 INFO DAGScheduler: Got map stage job 39 (count at cell49.sc:1) with 1 output partitions
24/09/06 14:50:58 INFO DAGScheduler: Final stage: ShuffleM

res49: Long = 2535L

In [50]:
(1 to 5).map(a => customerDf).reduce((x, y) => x.union(y)).count

24/09/06 14:50:59 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:59 INFO MemoryStore: Block broadcast_68 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:50:59 INFO MemoryStore: Block broadcast_68_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:50:59 INFO BlockManagerInfo: Added broadcast_68_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 

res50: Long = 2535L

### Join

In [51]:
val retailDf = spark.read.format("json").load("data/retail_data.json")

24/09/06 14:50:59 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/09/06 14:50:59 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
24/09/06 14:50:59 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:50:59 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:50:59 INFO MemoryStore: Block broadcast_75 stored as values in memory (estimated size 198.5 KiB, free 4.5 GiB)
24/09/06 14:50:59 INFO MemoryStore: Block broadcast_75_piece0 stored as bytes in memory (estimated size 33.9 KiB, free 4.5 GiB)
24/09/06 14:50:59 INFO BlockManagerInfo: Added broadcast_75_piece0 in memory on ubuntu:37607 (size: 33.9 KiB, free: 4.5 GiB)
24/09/06 14:50:59 INFO SparkContext: Created broadcast 75 from load at cell51.sc:1
24/09/06 14:50:59 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 bytes.
24/09/06 14:50:59 INFO SparkContext: Starting job: load at cell51.sc:1
24/09/06 14:50:59 INFO

retailDf: DataFrame = [CustomerID: string, Description: string ... 5 more fields]

In [52]:
customerDf.printSchema

root
 |-- Address: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Username: string (nullable = true)



In [53]:
retailDf.printSchema

root
 |-- CustomerID: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- InvoiceNo: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- UnitPrice: string (nullable = true)



In [54]:
(customerDf.dtypes.map(_._1)).toSet.intersect((retailDf.dtypes.map(_._1)).toSet)

res54: Set[String] = Set("CustomerID")

In [55]:
customerDf.join(retailDf, Seq("CustomerID")).show(10)

24/09/06 14:51:00 INFO FileSourceStrategy: Pushed Filters: IsNotNull(CustomerID)
24/09/06 14:51:00 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(CustomerID#179)
24/09/06 14:51:00 INFO FileSourceStrategy: Pushed Filters: IsNotNull(CustomerID)
24/09/06 14:51:00 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(CustomerID#865)
24/09/06 14:51:00 INFO CodeGenerator: Code generated in 4.797589 ms
24/09/06 14:51:00 INFO MemoryStore: Block broadcast_77 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:51:00 INFO MemoryStore: Block broadcast_77_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:51:00 INFO BlockManagerInfo: Added broadcast_77_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:51:00 INFO SparkContext: Created broadcast 77 from $anonfun$withThreadLocalCaptured$1 at FutureTask.java:264
24/09/06 14:51:00 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 byte

+----------+--------------------+-------------------+--------------+--------------------+---------------+----------------+--------------------+---------------+---------+--------+---------+---------+
|CustomerID|             Address|          Birthdate|       Country|               Email|           Name|        Username|         Description|    InvoiceDate|InvoiceNo|Quantity|StockCode|UnitPrice|
+----------+--------------------+-------------------+--------------+--------------------+---------------+----------------+--------------------+---------------+---------+--------+---------+---------+
|     12346|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|cooperalexis@hotm...|  Lindsay Cowan|valenciajennifer|MEDIUM CERAMIC TO...|1/18/2011 10:01|   541431|   74215|    23166|     1.04|
|     12346|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|cooperalexis@hotm...|  Lindsay Cowan|valenciajennifer|MEDIUM CERAMIC TO...|1/18/2011 10:17|  C541433|  -74215|    23166|     1.04|
|    

In [56]:
val newRetailDf = retailDf.withColumnRenamed("CustomerID", "rightCustomerID")

val customerRetailDf = customerDf.join(newRetailDf, customerDf("CustomerID") === newRetailDf("rightCustomerID"), "left")
customerRetailDf.show(10)

24/09/06 14:51:01 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:51:01 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:51:01 INFO FileSourceStrategy: Pushed Filters: IsNotNull(CustomerID)
24/09/06 14:51:01 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(CustomerID#865)
24/09/06 14:51:01 INFO CodeGenerator: Code generated in 6.81269 ms
24/09/06 14:51:01 INFO MemoryStore: Block broadcast_82 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:51:01 INFO MemoryStore: Block broadcast_82_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:51:01 INFO BlockManagerInfo: Added broadcast_82_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:51:01 INFO SparkContext: Created broadcast 82 from $anonfun$withThreadLocalCaptured$1 at FutureTask.java:264
24/09/06 14:51:01 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304 

+--------------------+-------------------+--------------+----------+--------------------+---------------+----------------+---------------+--------------------+---------------+---------+--------+---------+---------+
|             Address|          Birthdate|       Country|CustomerID|               Email|           Name|        Username|rightCustomerID|         Description|    InvoiceDate|InvoiceNo|Quantity|StockCode|UnitPrice|
+--------------------+-------------------+--------------+----------+--------------------+---------------+----------------+---------------+--------------------+---------------+---------+--------+---------+---------+
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|  Lindsay Cowan|valenciajennifer|          12346|MEDIUM CERAMIC TO...|1/18/2011 10:17|  C541433|  -74215|    23166|     1.04|
|Unit 1047 Box 408...|1994-02-20 00:46:27|United Kingdom|     12346|cooperalexis@hotm...|  Lindsay Cowan|valenciajennifer|          12346|ME

newRetailDf: DataFrame = [rightCustomerID: string, Description: string ... 5 more fields]
customerRetailDf: DataFrame = [Address: string, Birthdate: string ... 12 more fields]

In [57]:
customerRetailDf.select("CustomerID").show(10)

24/09/06 14:51:02 INFO FileSourceStrategy: Pushed Filters: 
24/09/06 14:51:02 INFO FileSourceStrategy: Post-Scan Filters: 
24/09/06 14:51:02 INFO FileSourceStrategy: Pushed Filters: IsNotNull(CustomerID)
24/09/06 14:51:02 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(CustomerID#865)
24/09/06 14:51:02 INFO CodeGenerator: Code generated in 3.295787 ms
24/09/06 14:51:02 INFO MemoryStore: Block broadcast_87 stored as values in memory (estimated size 198.4 KiB, free 4.5 GiB)
24/09/06 14:51:02 INFO MemoryStore: Block broadcast_87_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 4.5 GiB)
24/09/06 14:51:02 INFO BlockManagerInfo: Added broadcast_87_piece0 in memory on ubuntu:37607 (size: 34.0 KiB, free: 4.5 GiB)
24/09/06 14:51:02 INFO SparkContext: Created broadcast 87 from $anonfun$withThreadLocalCaptured$1 at FutureTask.java:264
24/09/06 14:51:02 INFO FileSourceScanExec: Planning scan with bin packing, max size: 4194304 bytes, open cost is considered as scanning 4194304

+----------+
|CustomerID|
+----------+
|     12346|
|     12346|
|     12347|
|     12347|
|     12347|
|     12347|
|     12347|
|     12347|
|     12347|
|     12347|
+----------+
only showing top 10 rows



## Stop

In [58]:
spark.stop()

24/09/06 14:51:02 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/09/06 14:51:02 INFO SparkUI: Stopped Spark web UI at http://ubuntu:4041
24/09/06 14:51:02 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/09/06 14:51:02 INFO MemoryStore: MemoryStore cleared
24/09/06 14:51:02 INFO BlockManager: BlockManager stopped
24/09/06 14:51:02 INFO BlockManagerMaster: BlockManagerMaster stopped
24/09/06 14:51:02 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/09/06 14:51:02 INFO SparkContext: Successfully stopped SparkContext
